In [5]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.offline as pyo
import plotly.io as pio
import pandas as pd
import numpy as np
import sys
sys.path.append('../src')

from data_collection import FinancialDataCollector
from data_processing import FinancialDataProcessor
from config import SECTOR_MAPPING

In [2]:
collector = FinancialDataCollector()
stock_data = collector.load_stock_data()
processor = FinancialDataProcessor(stock_data)

Data loaded from data\raw\stock_data.pkl


In [6]:
# Plot-1. Interactive Candlestick Chart
def create_interactive_candlestick():
    ticker = 'AAPL'
    df = stock_data[ticker]['prices'].reset_index()
    df['Date'] = pd.to_datetime(df['Date'])
    
    fig = go.Figure()
    
    fig.add_trace(go.Candlestick(                   # Add candlestick
        x=df['Date'],
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close'],
        name=ticker,
        increasing_line_color='green',
        decreasing_line_color='red'
    ))
    
    fig.add_trace(go.Scatter(               # Add moving averages
        x=df['Date'],
        y=df['MA_20'],
        mode='lines',
        name='MA 20',
        line=dict(color='blue', width=1)
    ))
    
    fig.add_trace(go.Scatter(
        x=df['Date'],
        y=df['MA_50'],
        mode='lines',
        name='MA 50',
        line=dict(color='orange', width=1)
    ))
    
    fig_volume = go.Figure()                # Add volume subplot
    fig_volume.add_trace(go.Bar(
        x=df['Date'],
        y=df['Volume'],
        name='Volume',
        marker_color='lightblue'
    ))
    
    fig.update_layout(                  # Update layout
        title=f'{ticker} Interactive Candlestick Chart',
        yaxis_title='Price ($)',
        xaxis_title='Date',
        template='plotly_white',
        height=600,
        xaxis_rangeslider_visible=True
    )
    
    fig.write_html(f'./static/plotly_{ticker}_candlestick.html')
    pio.show(fig, renderer='browser')
    
    return fig

candlestick_fig = create_interactive_candlestick()

In [7]:
# Plot-2. 3D Scatter Plot (Price, Volume, Market Cap)

def create_3d_scatter():
    scatter_data = []                       # Prepare data
    for ticker in list(stock_data.keys())[:15]:
        prices = stock_data[ticker]['prices']
        avg_price = prices['Close'].mean()
        avg_volume = prices['Volume'].mean()
        
        market_cap = stock_data[ticker]['info'].get('marketCap', 0) / 1e9  # # Get market cap from info in billions
        
        sector = 'Other'                    # Get sector
        for sect, tickers in SECTOR_MAPPING.items():
            if ticker in tickers:
                sector = sect
                break
        
        scatter_data.append({
            'Ticker': ticker,
            'Avg_Price': avg_price,
            'Avg_Volume': avg_volume / 1e6,     # In millions
            'Market_Cap': market_cap,
            'Sector': sector
        })
    
    scatter_df = pd.DataFrame(scatter_data)
    
    fig = px.scatter_3d(                        # Create 3D scatter plot
        scatter_df,
        x='Avg_Price',
        y='Avg_Volume',
        z='Market_Cap',
        color='Sector',
        size='Market_Cap',
        hover_name='Ticker',
        title='3D Analysis: Price vs Volume vs Market Cap',
        labels={
            'Avg_Price': 'Average Price ($)',
            'Avg_Volume': 'Average Volume (Millions)',
            'Market_Cap': 'Market Cap (Billions $)'
        }
    )
    
    fig.update_layout(height=600, template='plotly_white')
    fig.write_html('./static/plotly_3d_scatter.html')
    pio.show(fig, renderer='browser')
    
    return fig

scatter_3d_fig = create_3d_scatter()

In [8]:
# Plot-3. Interactive Time Series Dashboard
def create_time_series_dashboard():

    selected_stocks = ['AAPL', 'GOOGL', 'MSFT', 'TSLA', 'NVDA']     # Select stocks for dashboard
    
    fig = make_subplots(            # Create subplots
        rows=3, cols=1,
        shared_xaxes=True,
        subplot_titles=('Stock Prices', 'Daily Returns', 'Volatility'),
        vertical_spacing=0.05,
        specs=[[{"secondary_y": True}],
               [{"secondary_y": False}],
               [{"secondary_y": False}]]
    )
    
    colors = px.colors.qualitative.Set1
    
    for i, ticker in enumerate(selected_stocks):
        prices = stock_data[ticker]['prices']
        returns = prices['Returns'] * 100
        volatility = returns.rolling(window=30).std()
        
        normalized_prices = (prices['Close'] / prices['Close'].iloc[0]) * 100       # Normalize prices to start at 100
        
        fig.add_trace(          # Add price traces
            go.Scatter(
                x=prices.index,
                y=normalized_prices,
                mode='lines',
                name=f'{ticker} Price',
                line=dict(color=colors[i], width=2),
                visible=True if i < 3 else 'legendonly'
            ),
            row=1, col=1
        )
        
        fig.add_trace(              # Add returns traces
            go.Scatter(
                x=prices.index,
                y=returns,
                mode='lines',
                name=f'{ticker} Returns',
                line=dict(color=colors[i], width=1),
                showlegend=False,
                visible=True if i < 3 else 'legendonly'
            ),
            row=2, col=1
        )
        
        fig.add_trace(          # Add volatility traces
            go.Scatter(
                x=prices.index,
                y=volatility,
                mode='lines',
                name=f'{ticker} Volatility',
                line=dict(color=colors[i], width=1),
                showlegend=False,
                visible=True if i < 3 else 'legendonly'
            ),
            row=3, col=1
        )
    
    fig.update_layout(          # Update layout
        title='Interactive Stock Market Dashboard',
        height=800,
        template='plotly_white',
        hovermode='x unified'
    )
    
    # Update y-axis labels
    fig.update_yaxes(title_text="Normalized Price (Base=100)", row=1, col=1)
    fig.update_yaxes(title_text="Daily Returns (%)", row=2, col=1)
    fig.update_yaxes(title_text="30-Day Volatility (%)", row=3, col=1)
    fig.update_xaxes(title_text="Date", row=3, col=1)
    
    # Add range slider
    fig.update_layout(xaxis3=dict(rangeslider=dict(visible=True)))
    
    fig.write_html('./static/plotly_dashboard.html')
    pio.show(fig, renderer='browser')
    
    return fig

dashboard_fig = create_time_series_dashboard()

In [10]:
# Plot-4. Sector Performance Sunburst Chart
def create_sector_sunburst():
    
    sector_data = []                    # Calculate sector performance
    
    for sector, tickers in SECTOR_MAPPING.items():
        sector_returns = []
        sector_market_cap = 0
        
        for ticker in tickers:
            if ticker in stock_data:
                returns = stock_data[ticker]['prices']['Returns'].dropna()
                annual_return = returns.mean() * 252 * 100
                market_cap = stock_data[ticker]['info'].get('marketCap', 0) / 1e9
                
                sector_returns.append(annual_return)
                sector_market_cap += market_cap
                
                sector_data.append({                # Individual stock data
                    'ids': f"{sector}-{ticker}",
                    'labels': ticker,
                    'parents': sector,
                    'values': market_cap,
                    'annual_return': annual_return
                })
        
        if sector_returns:                              # Sector summary
            avg_sector_return = np.mean(sector_returns)
            sector_data.append({
                'ids': sector,
                'labels': f"{sector}<br>({len(sector_returns)} stocks)",
                'parents': "",
                'values': sector_market_cap,
                'annual_return': avg_sector_return
            })
    
    df = pd.DataFrame(sector_data)
    
    fig = go.Figure(go.Sunburst(
        ids=df['ids'],
        labels=df['labels'],
        parents=df['parents'],
        values=df['values'],
        branchvalues="total",
        hovertemplate='<b>%{label}</b><br>' +
                      'Market Cap: $%{value:.1f}B<br>' +
                      'Annual Return: %{customdata:.2f}%<extra></extra>',
        customdata=df['annual_return'],
        maxdepth=2,
    ))
    
    fig.update_layout(
        title="Market Sectors by Market Cap and Performance",
        height=600,
        template='plotly_white'
    )
    
    fig.write_html('./static/plotly_sunburst.html')
    pio.show(fig, renderer='browser')
    
    return fig

sunburst_fig = create_sector_sunburst()

In [11]:
# Plot-5. Correlation Heatmap with Plotly
def create_interactive_correlation():
    
    selected_stocks = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NVDA', 'JPM', 'JNJ', 'V']      # Get correlation matrix for selected stocks
    correlation_matrix = processor.calculate_correlation_matrix(selected_stocks)
    
    fig = go.Figure(data=go.Heatmap(
        z=correlation_matrix.values,
        x=correlation_matrix.columns,
        y=correlation_matrix.index,
        colorscale='RdBu',
        zmid=0,
        text=correlation_matrix.round(3).values,
        texttemplate="%{text}",
        textfont={"size": 10},
        hovertemplate='%{y} vs %{x}<br>Correlation: %{z:.3f}<extra></extra>'
    ))
    
    fig.update_layout(
        title='Interactive Stock Correlation Matrix',
        width=700,
        height=700,
        template='plotly_white'
    )
    
    fig.write_html('./static/plotly_correlation.html')
    pio.show(fig, renderer='browser')
    
    return fig

correlation_fig = create_interactive_correlation()